In [1]:
from collections import OrderedDict
from bs4 import BeautifulSoup
from pprint import pprint
from random import choice
from time import sleep
import urllib.request
import sqlite3
import tweepy
import praw
import sys
import os
#sys.path.append('/Users/Anthony/Desktop/python_projects_clean/reddit')
sys.path.append(r'C:\Users\Anthony\Documents\reddit')

In [2]:
import twitter_keys as tk
import reddit_keys as rk

def replacer(article_title, words_to_replace, random_nickname):
    """
    :param title: string you want to split.
    :param words: list of words you are looking to match.
    :param randoms: list of random words to replace with words.
    :return: new title
    """
    new_post = []
    rand_choice = []
    for word in posts[0].split():
        if word in trg_words:
            word = choice(trump_nicknames)
            rand_choice.append(word)
        new_post.append(word)

    return ' '.join(new_post), rand_choice

In [3]:
def reddit_login(reddit_api_keys):
    """API login info """

    login = praw.Reddit(user_name =reddit_api_keys[0],
                        password = reddit_api_keys[1], 
                        client_id = reddit_api_keys[2],
                        client_secret = reddit_api_keys[3],
                        user_agent = 'reddit twitter thing thing v1')
    return login


def tweepy_login(tweepy_api_keys):
    """ API login info """
    
    auth = tweepy.OAuthHandler(tweepy_api_keys[0], tweepy_api_keys[1])
    auth.set_access_token(tweepy_api_keys[2], tweepy_api_keys[3])
    return tweepy.API(auth)

def nicknames_file(path):
    """ path to file with nicknames for trump """

    nicknames = []
    with open(path, 'r') as f:
        for l in f:
            nn = l.replace('\n', '')
            nicknames.append(nn)
    return nicknames

def replacer(article_title, words_to_replace, random_nickname):
    """
    :param title: string you want to split.
    :param words: list of words you are looking to match.
    :param randoms: list of random words to replace with words.
    :return: new title
    """

    new_post = []
    for word in article_title.split():
        if word in words_to_replace:
            word = choice(random_nickname)
        new_post.append(word)

    return ' '.join(new_post)
    
    
    #return ' '.join([choice(random_nickname) if word in words_to_replace else word 
                        #for word in article_title.split()])


def create_db_table(full_path_to_db):
    """ create new DB """
    
    if os.path.exists(full_path_to_db):
        print(full_path_to_db + ' already exists')
    else:
        conn = sqlite3.connect(full_path_to_db)
        c = conn.cursor()
        c.execute("""CREATE TABLE reddit_posts_to_twitter
            (article_url text, 
            article_title text)""")

        conn.commit()
        conn.close()
        print('Created new database in ' + full_path_to_db)
    
    
def get_random_subreddit():
    """scrapes top 100 subreddits & excludes boring ones.  then chooses 1 at random """
    
    url = 'http://redditmetrics.com/top'
    
    r = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")

    body = soup.find_all('tbody')

    top_subreddits = []
    for i in body:
        trs = i.find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            for td in tds:
                texts = td.get_text()
                if '/r/' in texts:
                    top_subreddits.append(texts)
    
    exclude = ['gaming', 'DIY', 'nosleep', 'WritingPrompts', 'Fitness', 'leagueoflegends', 'listentothis', 'personalfinance',
               'gonewild', 'nsfw', 'history', '4chan', 'nba', 'NSFW_GIF', 'RealGirls', 'malefashionadvice', 'announcements',
               'TwoXChromosomes', 'GetMotivated', 'gameofthrones', 'europex', 'atheism', 'sex', 'OutOfTheLoop', 'Music', 'blog',
               'movies', 'videos', 'Overwatch', 'pokemongo', 'YouShouldKnow', 'wholesomememes', 'Showerthoughts']

    filtered_subreddits = [i[3:] for i in top_subreddits if i not in exclude]
    
    return choice(filtered_subreddits)
                    

def get_random_subreddit_top_posts(reddit_api_keys, random_subreddit):
    """ returns highest upvoted post in rising posts """
    
    r_login = reddit_login(reddit_api_keys)
    
    
    page = r_login.subreddit(random_subreddit)

    top_posts = page.hot(limit=100)

    posts_dict = {}
    print('this is', page, '\n\n')
    top_posts = page.hot(limit=20)
    for post in top_posts:
        if post.ups > 1000:
            value = post.ups, post.url
            posts_dict[post.title] = value

    # sort by most upvotes & get first entry         
    sorted_by_upvotes = OrderedDict(sorted(posts_dict.items(),key=lambda x:x[0], reverse=True))
    keys = list(sorted_by_upvotes)
    value = sorted_by_upvotes[keys[0]] # get first dict value 
    post_dict = {}
    post_dict[keys[0]] = value
    return post_dict


def get_politics_subreddit(r_login, subreddit, trump_nicknames, dict_length):
    """returns dict of tuples of url, upvotes, title"""
    
    page = r_login.subreddit(subreddit)
    
    # different types of apostrophes :'(
    words_to_replace = ["President Trump", "Donald Trump", "Trump", "POTUS",  
                         "Trump’s", "Trump's", "President Trump’s", "President Trump's", "Donald Trump’s", "Donald Trump's"]
    
    irrelevant = ["Trumpcare", "Trump-care", "Trump-Russia", "Melania", "Ivanka"]

    nicknames_apostrophe = [i + "'s" for i in trump_nicknames]

    post_dict = {}
    top_posts = page.hot(limit=100)
    for post in top_posts:
        title = post.title
        if any(word in title for word in irrelevant):
            pass
        elif any(word in title for word in words_to_replace):
            replace_apost = replacer(post.title, words_to_replace[4:], nicknames_apostrophe)
            replace_no_apost = replacer(replace_apost, words_to_replace[:4], trump_nicknames)
            if post.ups > 1000: 
                value = post.ups, post.url
                post_dict[replace_no_apost] = value
                
    return shorten_posts_dict(post_dict, dict_length)

def shorten_posts_dict(posts_dict, int_length_of_dict):
    '''not used with random subreddits cuz we only want 1 from each subreddit'''

    sorted_by_upvotes = OrderedDict(sorted(posts_dict.items(),key=lambda x:x[1], reverse=True))

    keys = list(sorted_by_upvotes)


    top_upvotes_dict = {}
    for i in range(int_length_of_dict):
        values = sorted_by_upvotes[keys[i]]
        top_upvotes_dict[keys[i]] = values
    return top_upvotes_dict


def post_tweet(tweepy_keys, tweet_dict, db_path):
    """takes reddit post dict of tuples, write to DB if not there & posts to twitter"""

    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    for k,v in tweet_dict.items():
        try:
            c.execute('SELECT * FROM reddit_posts_to_twitter WHERE (article_url=? AND article_title=?)', (k, v[1]))
            entry = c.fetchone()

            if entry is None:
                c = conn.cursor()
                c.execute("""
                    INSERT INTO 
                        reddit_posts_to_twitter 
                    VALUES
                        (?, ?)
                """, (k, v[1]))
                conn.commit()
                tweet = k + '\n'+ v[1]
                api = tweepy_login(tweepy_keys)
                api.update_status(tweet)
                sleep(5)
                print ('\n', 'New entry added', '\n', k, v[1], '\n')
            else:
                print ('Entry found')
        except Exception as e:
            print(str(e))
            
def reddit_tweeter(tweepy_api_keys, reddit_api_keys, db_path, int_tweets_to_tweet, 
                   subreddit_name=None, nicknames_path=None, random=False):
    """ have choice of random subreddit tweeter or trump news tweeter """
    
    r_login = reddit_login(reddit_api_keys)
    
    if random:
        count=0
        while count < int_tweets_to_tweet:
            count+=1
            random_subreddit = get_random_subreddit()
            print('Checking DB for duplicates...')
            posts_dict = get_random_subreddit_top_posts(reddit_api_keys, random_subreddit)
            print(count, post_tweet(tweepy_api_keys, posts_dict, db_path))
            
    if not random: 
        nicknames = nicknames_file(nicknames_path)
        print('Checking DB for duplicates...')
        posts_dict = get_politics_subreddit(r_login, subreddit_name, nicknames, int_tweets_to_tweet)
        print(post_tweet(tweepy_api_keys, posts_dict, db_path))

In [4]:
tweepy_api_keys = tk.CONSUMER_KEY, tk.CONSUMER_SECRET, tk.ACCESS_TOKEN, tk.ACCESS_TOKEN_SECRET
reddit_api_keys = rk.username, rk.password, rk.client_id, rk.client_secret
db_path = r'C:\Users\Anthony\Documents\reddit\trump_db1.sqlite'

# Create new database 

In [79]:
create_db_table(db_path)

Created new database in C:\Users\Anthony\Documents\reddit\trump_db1.sqlite


# Random subreddit tweet

In [7]:
reddit_tweeter(tweepy_api_keys, reddit_api_keys, db_path, 5, random=True)

Checking DB for duplicates...
this is UpliftingNews 


[{'message': 'Status is over 140 characters.', 'code': 186}]
1 None
Checking DB for duplicates...
this is Frugal 



 New entry added 
 [Suggestion] How about, instead of telling us about the deal you got - you tell us how you did it? https://www.reddit.com/r/Frugal/comments/71dy9s/suggestion_how_about_instead_of_telling_us_about/ 

2 None
Checking DB for duplicates...
this is RealGirls 




KeyboardInterrupt: 

# Trump nicknames tweet

In [5]:
#nicknames = '/Users/Anthony/Desktop/python_projects_clean/reddit/nicknames.txt'
nn_path = r'C:\Users\Anthony\Documents\reddit\nicknames.txt'

db_path = r'C:\Users\Anthony\Documents\reddit\trump_db1.sqlite'

reddit_tweeter(tweepy_api_keys, reddit_api_keys, db_path, 3, 
               subreddit_name= 'politics', nicknames_path=nn_path)

Checking DB for duplicates...

 New entry added 
 Cinnamon Hitler praises health care of Nambia, a nonexistent African country http://www.cnn.com/2017/09/21/africa/trump-nambia-un-africa-trnd/index.html 


 New entry added 
 Manafort used The Presumptuous Nominee campaign account to email Ukrainian operative http://www.politico.com/story/2017/09/20/paul-manafort-trump-campaign-email-ukrainian-operative-242949 


 New entry added 
 Fox News Slammed by Legal Experts for Saying Humpty Trumpty Was Right About Obama Wiretap http://www.newsweek.com/fox-news-slammed-legal-experts-trump-right-obama-wiretap-668117?utm_campaign=NewsweekTwitter&utm_source=Twitter&utm_medium=Social 

None


In [ ]:
random_subreddit =  get_random_subreddit()
get_random_subreddit_top_posts(reddit_api_keys, random_subreddit)

# Find more twitter followers 

In [167]:
for idx,tweet in enumerate(tweepy.Cursor(api.search, q='#politics').items()):
    if tweet.author.followers_count > 500:
        api.create_friendship(tweet.author.id)
        sleep(4)
        print("{} || Following @{}".format(idx, api.get_user(tweet.author.id).screen_name) + '|' + tweet.text) 

5 || Following @brexit_politics|Forget gaining £350m a week, #Brexit would cost the UK £300m a week https://t.co/UVQbcloQyH #BBC #News #Politics
6 || Following @brexit_politics|Theresa May will get no sympathy for #Brexit in Eurosceptic Italy https://t.co/c44SyY8zHk #BBC #News #Politics
7 || Following @dailyurdunewspk|#DailyUrduNews #Urdu #UrduNews #Urdunews-  Sheikh Rasheed Per Bijlian  - https://t.co/9EJxwHOAsI - #Pakistan… https://t.co/uuh5vNWvfP
9 || Following @olwage|The US, climate change and the farmers of Nepal https://t.co/85Vw6UGlf9 #climate #Nepal #politics #USA #agriculture
11 || Following @camaripop|Thought Factory: #RadicalEconomicTransformation: Spin or Radical Agenda? #politics #SouthAfrica ɱ13 https://t.co/qiZrfpovO9
12 || Following @bctallis|RT @CEENewP: Great new piece by @claudia_aradau of @warstudies  responding to Peter #Sloterdijk  - Performative #Politics and #IR https://t…
14 || Following @INDIEBOOKSOURCE|RT @jackieranke19: #badpresident #yourefired #politics h

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/users/show.json?id=1584343075 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x11524a6a0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))